In [ ]:
{
    "query": "다섯 명의 여성이 해변을 따라 플립플롭을 신고 걸어간다.", # 질의문
    "pos": ["플립플롭을 신은 몇몇 여성들이 해변을 따라 걸어가고 있다"], # 검색결과 (연관된 문서/문장)
    "neg": [ # 연관 없는 문서/문장
        "4명의 여성이 해변에 앉아 있다.",
        "1996년에 개혁이 있었다.",
        "그녀는 자신의 기록을 정정하기 위해 법정에 가지 않을 것이다.",
        "그 남자는 하와이에 대해 이야기하고 있다.",
        "한 여성이 밖에 서 있다.",
        "전투는 끝났다.",
        "한 무리의 사람들이 배구를 하고 있다.",
    ],
}

## Train dataset

In [26]:
import pandas as pd

fpath = "../data/sts-train.tsv"
try:
    file = pd.read_csv(fpath, sep='\t', on_bad_lines='skip')
except pd.errors.ParserError as e:
    print(e)
    
len(file)

5696

In [27]:
## score 2.5 이상
pos = file.loc[file['score'] > 2.5].sort_values('score')

print(len(pos))
pos[pos.isna().any(axis=1)]

3343


,genre,filename,year,id,score,sentence1,sentence2
2475,main-news,MSRpar,2012test,49,3.2,나에게 르윈스키 임브로글리오는 어제 발표된 추출물에 따르면 정치적 반대자들이 제작한...,NaN
3294,main-news,MSRpar,2012train,585,3.6,캡스 2 시스템의 임무는 항상 항공 보안이었고 항상 국토 안보부의 일부인 행정부가 ...,NaN
2874,main-news,MSRpar,2012test,684,4.0,처음부터 우리는 아무도 임금 삭감을 받는 것을 보고 싶지 않았습니다.\t그러나 크로...,NaN


In [28]:
is_na = []

for i, row in pos[pos.isna().any(axis=1)].iterrows():
    print(i)
    is_na.append(i)
    s = row['sentence1'].split("\t")
    print(s)
    pos.loc[i, 'sentence1'] = s[0]
    pos.loc[i, 'sentence2'] = s[1]

print("After")

pos.loc[is_na]

2475
['나에게 르윈스키 임브로글리오는 어제 발표된 추출물에 따르면 정치적 반대자들이 제작한 또 다른 악의적인 스캔들처럼 보였다.', '나에게 르윈스키 임브로글리오는 정치적 반대자들에 의해 제조된 또 다른 악의적인 스캔들처럼 보였다."클린턴 여사는 썼다.']
3294
['캡스 2 시스템의 임무는 항상 항공 보안이었고 항상 국토 안보부의 일부인 행정부가 말했다.', '캡스 2 시스템의 임무는 항공 보안이었고 항상 항공 보안이 될 것입니다."라고 그들은 말했다.']
2874
['처음부터 우리는 아무도 임금 삭감을 받는 것을 보고 싶지 않았습니다.', '그러나 크로스비 씨는 관련 언론에 처음부터 우리는 아무도 임금 삭감을 받는 것을 보고 싶지 않았다.']
After


,genre,filename,year,id,score,sentence1,sentence2
2475,main-news,MSRpar,2012test,49,3.2,나에게 르윈스키 임브로글리오는 어제 발표된 추출물에 따르면 정치적 반대자들이 제작한...,나에게 르윈스키 임브로글리오는 정치적 반대자들에 의해 제조된 또 다른 악의적인 스캔...
3294,main-news,MSRpar,2012train,585,3.6,캡스 2 시스템의 임무는 항상 항공 보안이었고 항상 국토 안보부의 일부인 행정부가 ...,"캡스 2 시스템의 임무는 항공 보안이었고 항상 항공 보안이 될 것입니다.""라고 그들..."
2874,main-news,MSRpar,2012test,684,4.0,처음부터 우리는 아무도 임금 삭감을 받는 것을 보고 싶지 않았습니다.,그러나 크로스비 씨는 관련 언론에 처음부터 우리는 아무도 임금 삭감을 받는 것을 보...


In [29]:
print(len(pos))

pos[pos.isna().any(axis=1)]

3343


,genre,filename,year,id,score,sentence1,sentence2


In [30]:
queries = []

for i, row in pos.iterrows():
    query = {
        "query" : row["sentence1"],
        "pos" : [row["sentence2"]]
    }
    queries.append(query)
    

queries

[{'query': '한 여성이 팬케이크를 뒤집고 있다.', 'pos': ['만화 요리사가 팬케이크를 뒤집고 있다.']},
 {'query': '주방장이 수프 그릇을 치우고 있다.', 'pos': ['남자가 그릇 가장자리를 닦고 있다.']},
 {'query': '남자가 종이를 자르고 있다.', 'pos': ['사람이 종이를 찢고 있다.']},
 {'query': '본드 불스는 연방준비제도이사회가 위험이 경제적 약세로 치우쳐 있다는 것을 인식하기를 바란다.',
  'pos': ['연방준비제도이사회는 또한 경제에 대한 위험은 취약성에 치우쳐 있다고 말했다.']},
 {'query': '파키스탄 북부에서 무장괴한들이 외국인 관광객 10명을 살해하다',
  'pos': ['이슬람 무장단체는 9명의 외국인 관광객, 1명의 파키스탄인을 살해한다']},
 {'query': '노조는 또한 3월에 5일간의 파업을 벌여 예일 식당 중 하나를 제외한 모든 사람들을 폐쇄시켰다.',
  'pos': ['노조는 또한 3월에 5일간의 파업을 벌였다.파업은 지난 10건의 계약 중 8건을 앞섰다.']},
 {'query': '남자가 말을 타고 있다.', 'pos': ['한 소녀가 말을 타고 있다.']},
 {'query': '딘은 10개 도시 "잠 못 이루는 여름" 투어를 통해 기자들에게 텍사스에서의 선거운동을 도전으로 간주했다고 말했다.',
  'pos': ['오늘, 딘은 시카고와 뉴욕에서 4일간의 "잠 못 이루는 여름"투어를 끝냅니다.']},
 {'query': '리는 시리아에서 화학 무기 사용을 비난한다.', 'pos': ['시리아의 화학 무기 주장에 대한 의문이 제기된다.']},
 {'query': '드조카르 차르나예프는 보스턴 폭탄테러 혐의에 대해 무죄를 주장했다.',
  'pos': ['보스턴 마라톤 폭탄테러 혐의로 기소된 드조카르 차르네프']},
 {'query': '아프가니스탄 국회의원들은 새로운 선거법을 승인한다.', 'pos': ['아프가니스탄 대통령, 새로운 선거법 승

In [31]:
import json
# JSONL 파일로 저장
with open('../data/ft_data.jsonl', 'w', encoding='utf-8') as f:
    for item in queries:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

테스트 코드

In [ ]:
import json

corpus = []
queries = []
train_data = []
for line in open('../data/ft_data.jsonl'):
    line = json.loads(line.strip())
    # print(line["pos"])
    # break
    train_data.append(line)
    corpus.extend(line["pos"])
    if "neg" in line:
        corpus.extend([line["neg"]])
    queries.append(line["query"])


In [ ]:
for c in corpus:
    if not isinstance(c, str):
        print(type(c))
        print(c)

In [ ]:
import pandas as pd
path = '../data/ft_data_minedHN.jsonl'
tune_df = pd.read_json(path, lines=True)

print(max(tune_df['query'].str.len()))

max_len = 0
for corpus in tune_df['pos']:
    c_l = 0
    for s in corpus:
        c_l += len(s)
    if (c_l > max_len) : 
        print(corpus)
        max_len = c_l

print("Pos 최대 바이트 길이이 : ", max_len)

max_len = 0
for corpus in tune_df['neg']:
    c_l = 0
    for s in corpus:
        c_l += len(s)
    if (c_l > max_len) : 
        print(corpus)
        max_len = c_l

print("Neg 최대 바이트 길이: ", max_len)

98
['터키 대통령은 전국적인 시위 속에서 침착함을 요구한다.']
['나무 테이블에는 양초, 와인, 그리고 보라색 플라스틱 그릇이 놓여 있습니다.']
['업그레이드는 2004년 1월에 스마넷 지원을 받는 현재 고객들에게 무료 다운로드로 제공될 것입니다.']
['유카이파는 1995년에 도미닉을 6억 9천 3 백만 달러에 사서 1998년에 18억 달러에 Safeway에 팔았습니다.']
['달러화는 4월 30일 이후 가장 강력한 수준으로 상승하기 위해 화요일의 1% 반등을 연장하면서, 초기 거래에서 유로화에 1.1115달러까지 상승했다.']
['볼리비아의 군대는 수요일에 다이너마이트를 휘두르는 광부들이 포위된 수도로 흘러들어가는 것을 막기 위해 싸웠고, 대통령에 대한 대중 봉기로 두 명이 사망했다.']
['일본은 어제 버마에 대한 국제사회의 압력에 추가하여, 군부가 친 민주주의 지도자 아웅산 수 지를 석방하지 않는 한, 막대한 경제 원조를 중단하겠다고 위협하고 있다.']
['Nvidia는 Siemens ag, sharp, philips, dell, mitsubishi, 소니 코퍼레이션과 같은 플레이어를 포함하는 mediaq의 고객을 이용할 것입니다.']
['브레머는 이라크 전역의 이웃들을 청소하고 지역 사회 프로젝트를 건설하기 위해 앞으로 2주 동안 7천만 달러 규모의 프로그램을 시작할 것이라고 말했지만 자세한 내용은 밝히지 않았다.']
Pos 최대 길이 :  100
['67세의 주교인 O Brien은 경찰에게 그가 개나 고양이를 쳤다고 생각했다고 말했다.', '한 남자와 여자가 함께 영화를 본다.', '이 질문에 제가 답하겠습니다', '이라크, 시리아행 이란 비행기 수색', '나는 그것이 나쁜 생각이라고 생각한다.', '이것은 큰 문제이다.', '사우디 아라비아, 유엔 안전보장이사회 의석 거부', '당신에게 두 가지 제안을 드리겠습니다', '이게 당신의 질문에 답해주나요?', "파키스탄은 '신성모독'에 대해 트위터를 차단한다.", '당신의 두 질문에 대한 답은 

## Test dataset

In [47]:
import pandas as pd

path = "../data/sts-test.tsv"
test = pd.read_csv(path, sep='\t')

test[test['sentence2'].isna()].index # NaN

Index([989, 1051, 1069], dtype='int64')

In [48]:
test

,genre,filename,year,id,score,sentence1,sentence2
0,main-captions,MSRvid,2012test,24,2.5,한 소녀가 머리를 스타일링하고 있다.,한 소녀가 머리를 빗고 있다.
1,main-captions,MSRvid,2012test,33,3.6,한 무리의 남자들이 해변에서 축구를 한다.,한 무리의 소년들이 해변에서 축구를 하고 있다.
2,main-captions,MSRvid,2012test,45,5.0,한 여성이 다른 여성의 발목을 재고 있다.,한 여자는 다른 여자의 발목을 측정한다.
3,main-captions,MSRvid,2012test,63,4.2,한 남자가 오이를 자르고 있다.,한 남자가 오이를 자르고 있다.
4,main-captions,MSRvid,2012test,66,1.5,한 남자가 하프를 연주하고 있다.,한 남자가 키보드를 연주하고 있다.
...,...,...,...,...,...,...,...
1374,main-news,headlines,2016,1354,0.0,필리핀과 캐나다는 더 나아가 관계를 증진시키겠다고 약속한다.,필리핀은 여객선 침몰 후 100명을 구한다.
1375,main-news,headlines,2016,1360,1.0,이스라엘은 팔레스타인인의 예루살렘의 구 시가지 출입을 금지한다.,팔레스타인인과 이스라엘 두 국가 해결안은 그림의 떡
1376,main-news,headlines,2016,1368,1.0,비밀경호국에 대해 얼마나 알고 계십니까?,양측 의원들은 비밀경호국에 분노를 표출한다
1377,main-news,headlines,2016,1420,0.0,이란 회담이 재개되면서 오바마는 사우디의 두려움을 달래기 위해 고군분투하고 있다.,미얀마는 일요일 투표를 위한 유권자 명단을 확정하기 위해 고군분투하고 있다.


In [49]:
test = test[test.loc[:, 'score'] >= 2.5]

In [50]:
for i, row in test.iterrows():
    if "\t" in row['sentence1']:
        print(i)
        chunks = row['sentence1'].split("\t")
        print(chunks)
        test.loc[i, 'sentence1'] = chunks[0]
        test.loc[i, 'sentence2'] = chunks[1]

# test['sentence2'] = test['sentence1'].apply(lambda s : s.split("\t")[1] if "\t" in s else test['sentence2'])

test[test['sentence2'].isna()].index # NaN

989
["그것은 '어이없다,' funny Cide의 트레이너 Barclay tagg는 말했다.", '한편, funny cide의 트레이너인 바클레이 태그는 그 주장을 우스꽝스럽다"라고 불렀다.']
1051
['우리는 우리의 패치 프로세스를 개선하기 위해 많은 노력과 에너지를 쏟아붓습니다. 아마도 우리가 했어야 했던 것보다 늦게 그리고 이제 우리는 믿을 수 없는 속도를 얻고 있습니다.', '우리는 패치 진행을 개선하기 위해 많은 노력과 에너지를 쏟아부었습니다. 아마도 우리가 해야 했던 것보다 늦게.']
1069
['내일 미션 여관에서 나는 위대한 캘리포니아 주의 주지사 당선인을 축하할 기회가 있습니다.', '나는 위대한 캘리포니아 주의 주지사 당선자를 축하할 기회가 있으며, 나는 그것을 기대하고 있다."']


Index([], dtype='int64')

In [51]:
test

,genre,filename,year,id,score,sentence1,sentence2
0,main-captions,MSRvid,2012test,24,2.5,한 소녀가 머리를 스타일링하고 있다.,한 소녀가 머리를 빗고 있다.
1,main-captions,MSRvid,2012test,33,3.6,한 무리의 남자들이 해변에서 축구를 한다.,한 무리의 소년들이 해변에서 축구를 하고 있다.
2,main-captions,MSRvid,2012test,45,5.0,한 여성이 다른 여성의 발목을 재고 있다.,한 여자는 다른 여자의 발목을 측정한다.
3,main-captions,MSRvid,2012test,63,4.2,한 남자가 오이를 자르고 있다.,한 남자가 오이를 자르고 있다.
6,main-captions,MSRvid,2012test,76,3.5,한 남자가 전기 자전거를 타고 있다.,한 남자가 자전거를 타고 있다.
...,...,...,...,...,...,...,...
1357,main-news,headlines,2016,405,5.0,Mt Gox는 20만 개의 '잊혀진' 비트코인을 밝힌다.,Mt Gox는 20만 개의 '잊혀진' 비트코인을 발견한다.
1358,main-news,headlines,2016,471,4.0,텍사스 공장 폭발로 100명 이상이 부상당했다.,수백 명의 사람들이 텍사스 비료 공장 폭발로 부상당한 것으로 추정된다.
1361,main-news,headlines,2016,735,4.0,보도에 따르면 남한 군인은 북한과의 국경에서 5명의 동지들을 살해한 후 도망친다고 한다.,남한 군인이 동지들을 죽이고 도망친다.
1364,main-news,headlines,2016,904,5.0,애비는 세금 변경 후 550억 달러 샤이어 거래에 관심을 잃는다.,애비는 미국 세금 변경 후 550억 달러 샤이어 거래에 관심이 식는다.


In [ ]:
queries = test[['id', 'sentence1']]
queries = queries.rename(columns={'sentence1' : 'text'})

# 기존 ID 는 전체 문서 내에서 유일하지 않음
# print(queries[queries['id'].duplicated()] )

queries.loc[:, 'id'] = [i for i in range(len(queries))]

# queries.set_index('id', inplace=True)
# queries.reset_index(inplace=True)
queries

,id,text
0,0,한 소녀가 머리를 스타일링하고 있다.
1,1,한 무리의 남자들이 해변에서 축구를 한다.
2,2,한 여성이 다른 여성의 발목을 재고 있다.
3,3,한 남자가 오이를 자르고 있다.
4,4,한 남자가 전기 자전거를 타고 있다.
...,...,...
767,767,Mt Gox는 20만 개의 '잊혀진' 비트코인을 밝힌다.
768,768,텍사스 공장 폭발로 100명 이상이 부상당했다.
769,769,보도에 따르면 남한 군인은 북한과의 국경에서 5명의 동지들을 살해한 후 도망친다고 한다.
770,770,애비는 세금 변경 후 550억 달러 샤이어 거래에 관심을 잃는다.


In [67]:
corpus = test[['id', 'sentence2']]
corpus = corpus.rename(columns={'sentence2': 'text'})

corpus['text'] = corpus['text'].apply(lambda x : [x] if not isinstance(x, list) else x)

corpus.loc[:, 'id'] = [i for i in range(len(corpus))]

corpus.set_index('id', inplace=True)
corpus.reset_index(inplace=True)

corpus

,id,text
0,0,[한 소녀가 머리를 빗고 있다.]
1,1,[한 무리의 소년들이 해변에서 축구를 하고 있다.]
2,2,[한 여자는 다른 여자의 발목을 측정한다.]
3,3,[한 남자가 오이를 자르고 있다.]
4,4,[한 남자가 자전거를 타고 있다.]
...,...,...
767,767,[Mt Gox는 20만 개의 '잊혀진' 비트코인을 발견한다.]
768,768,[수백 명의 사람들이 텍사스 비료 공장 폭발로 부상당한 것으로 추정된다.]
769,769,[남한 군인이 동지들을 죽이고 도망친다.]
770,770,[애비는 미국 세금 변경 후 550억 달러 샤이어 거래에 관심이 식는다.]


In [68]:
qrels = pd.DataFrame()
qrels.loc[:, 'qid'] = queries.loc[:, 'id']
qrels.loc[:, 'docid'] = corpus.loc[:, 'id']
qrels.loc[:, 'relevance'] = [1] * len(queries)

qrels

,qid,docid,relevance
0,0,0,1
1,1,1,1
2,2,2,1
3,3,3,1
4,4,4,1
...,...,...,...
767,767,767,1
768,768,768,1
769,769,769,1
770,770,770,1


In [69]:
import json

def save_jsonl(path : str, df : pd.DataFrame):
    l = []
    cols = df.columns
    
    for idx in range(len(df)):
        frame = {}
        for c in cols:
            if isinstance(df.loc[idx, c], list):
                frame[c] = df.loc[idx, c]
            elif c == 'relevance':
                frame[c] = int(df.loc[idx, c])
            else:
                frame[c] = str(df.loc[idx, c])
        l.append(frame)

    # JSONL 파일로 저장
    with open(path, 'w', encoding='utf-8') as f:
        for item in l:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')

save_jsonl("../data/test_queries.jsonl", queries)
save_jsonl("../data/test_corpus.jsonl", corpus)
save_jsonl("../data/test_qrels.jsonl", qrels)

## 테스트 코드

In [26]:
from datasets import load_dataset

queries = load_dataset("json", data_files="../data/test_queries.jsonl")['train']
corpus = load_dataset("json", data_files="../data/test_corpus.jsonl")['train']
qrels = load_dataset("json", data_files="../data/test_qrels.jsonl")['train']

print(queries)
print(corpus)
print(qrels)

d:\anaconda3\envs\lm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 1379 examples [00:00, 13100.97 examples/s]
Generating train split: 1379 examples [00:00, 92749.40 examples/s]
Generating train split: 1379 examples [00:00, 167050.17 examples/s]

Dataset({
    features: ['id', 'text'],
    num_rows: 1379
})
Dataset({
    features: ['id', 'text'],
    num_rows: 1379
})
Dataset({
    features: ['qid', 'docid', 'relevance'],
    num_rows: 1379
})


In [ ]:
queries_text = queries["text"]
corpus_text = [text for sub in corpus["text"] for text in sub]

qrels_dict = {}
for line in qrels:
    if line["qid"] not in qrels_dict:
        qrels_dict[line["qid"]] = {}
    qrels_dict[line["qid"]][line["docid"]] = line["relevance"]

print(len(corpus_text))
print(len(queries_text))

1379
1379


In [ ]:
with open("test_corpus_text.txt", 'w', encoding='utf-8') as f:
    for t in corpus_text:
        f.write(t + "\n")

In [70]:
for i in range(len(corpus_text)):
    if (corpus_text[i] != corpus[i]['text'][0]):
        print(f"{i} : {corpus_text[i]}")
        print(f"{i} : {corpus[i]}")
        break

990 : 한편, funny cide의 트레이너인 바클레이 태그는 그 주장을 우스꽝스럽다"라고 불렀다.
990 : {'id': '990', 'text': ['작년에 쿠바 관리들은 약 20명의 후보로 지명된 쿠바 음악가들 중 누구도 로스 엔젤레스의 그래미에 참석하기 위해 미국 비자를 제때 받지 못했다고 말했다.']}


In [28]:
df_minedHN = load_dataset('json', data_files='../data/ft_data_minedHN.jsonl')['train']

df = load_dataset('json', data_files='../data/ft_data.jsonl')['train']

print(df_minedHN)
print(df)

Dataset({
    features: ['query', 'pos', 'neg'],
    num_rows: 3343
})
Dataset({
    features: ['query', 'pos'],
    num_rows: 3343
})


In [29]:
import pandas as pd

df = pd.read_json("../data/ft_data_minedHN.jsonl", lines=True)

print(df.loc[:, 'pos'].isna().any())

len(df)

False


3343